## 1/2 -- Inputar dados do MapBiomas na malha de setores censitários

### A) Exposição a deslizamentos -- Área afetada -- Converter para EPSG 5880

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# --- CONFIGURAÇÃO ---
input_gpkg = '../data/clean/BR_setores_CD2022_e5_v1234_i1234_norm.gpkg' 
raster_deslizamentos = '../data/raw/mapbiomas/mapbiomas_landslides_2024.tif'
output_step1 = '../data/raw/mapbiomas/temp_step1_deslizamentos_5880.gpkg'

# EPSG Desejado para o arquivo final (Brazil Polyconic)
target_epsg = 5880

print("1. Carregando malha de setores...")
gdf = gpd.read_file(input_gpkg)
print(f"   CRS original do vetor: {gdf.crs}")

# --- AJUSTE DE CRS PARA CÁLCULO ---
print("2. Verificando CRS do Raster e alinhando vetor...")
with rasterio.open(raster_deslizamentos) as src:
    raster_crs = src.crs
    print(f"   CRS do Raster: {raster_crs}")
    
    # Reprojetar o vetor para o MESMO sistema do raster temporariamente
    # Isso é obrigatório para o zonal_stats funcionar
    gdf_calc = gdf.to_crs(raster_crs)

# --- CÁLCULO (ZONAL STATS) ---
print("3. Calculando estatísticas zonais (Deslizamentos)...")
# stats='mean' em um raster binário (0 e 1) retorna a porcentagem de área coberta (0 a 1)
stats = zonal_stats(gdf_calc, raster_deslizamentos, stats=['mean'])
df_stats = pd.DataFrame(stats)

# Preencher nulos com 0 (caso o setor não toque no raster)
df_stats['mean'] = df_stats['mean'].fillna(0)

# Passar os dados de volta para o GeoDataFrame original (ou para o reprojetado, tanto faz)
# Vamos usar o gdf_calc e depois converter para o final
gdf_calc['e_des_pct'] = df_stats['mean'] 

# --- NORMALIZAÇÃO ---
print("4. Normalizando indicador...")
scaler = MinMaxScaler(feature_range=(0, 1))
gdf_calc['e_des_norm'] = scaler.fit_transform(gdf_calc['e_des_pct'].values.reshape(-1, 1))

# --- CONVERSÃO FINAL PARA EPSG 5880 E SALVAMENTO ---
print(f"5. Convertendo para EPSG:{target_epsg} e salvando...")
gdf_final = gdf_calc.to_crs(epsg=target_epsg)

gdf_final.to_file(output_step1, driver='GPKG')
print("Concluído Código 1.")

1. Carregando malha de setores...
   CRS original do vetor: EPSG:4674
2. Verificando CRS do Raster e alinhando vetor...
   CRS do Raster: EPSG:4326
3. Calculando estatísticas zonais (Deslizamentos)...
4. Normalizando indicador...
5. Convertendo para EPSG:5880 e salvando...
Concluído Código 1.


### B) Exposição a Inundações - Ponderado pelo número de domicílios -- Converter para EPSG 5880

In [ ]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# --- CONFIGURAÇÃO ---
input_gpkg = '../data/raw/mapbiomas/temp_step1_deslizamentos_5880.gpkg' 
raster_inundacoes = '../data/raw/mapbiomas/mapbiomas_floods_2024.tif'
output_gpkg_final = '../data/clean/BR_setores_CD2022_e125_v1234_i1234_norm.gpkg'
target_epsg = 5880 # Brazil Polyconic

print("1. Carregando malha do passo anterior...")
gdf = gpd.read_file(input_gpkg)

# --- AJUSTE DE CRS PARA CÁLCULO ---
print("2. Verificando CRS do Raster de Inundações...")
with rasterio.open(raster_inundacoes) as src:
    raster_crs = src.crs
    print(f"   CRS do Raster: {raster_crs}")
    
    # O arquivo de entrada está em 5880, precisamos converter para o do raster (ex: 4326)
    # para o cálculo geométrico bater
    gdf_calc = gdf.to_crs(raster_crs)

# --- CÁLCULO (ZONAL STATS) ---
print("3. Calculando área inundada...")
stats_floods = zonal_stats(gdf_calc, raster_inundacoes, stats=['mean'])
df_floods = pd.DataFrame(stats_floods)
df_floods['mean'] = df_floods['mean'].fillna(0)

gdf_calc['e_inu_pct'] = df_floods['mean']

# --- PONDERAÇÃO POR DOMICÍLIOS ---
# Importante: Como 'V00001' é um número absoluto (contagem), ele independe do CRS.
coluna_total_domicilios = 'V00001' 

if coluna_total_domicilios in gdf_calc.columns:
    print("4. Ponderando pela densidade de domicílios...")
    # % da área alagada * Quantidade de casas no setor = Estimativa de casas atingidas
    gdf_calc['e_inu_abs'] = gdf_calc['e_inu_pct'] * gdf_calc[coluna_total_domicilios]
else:
    print(f"ATENÇÃO: Coluna {coluna_total_domicilios} não encontrada. Usando valor bruto.")
    gdf_calc['e_inu_abs'] = gdf_calc['e_inu_pct']

# --- NORMALIZAÇÃO ---
print("5. Normalizando indicador de inundações...")
scaler = MinMaxScaler(feature_range=(0, 1))
gdf_calc['e_inu_norm'] = scaler.fit_transform(gdf_calc['e_inu_abs'].values.reshape(-1, 1))

# --- FINALIZAÇÃO ---
print(f"6. Revertendo para EPSG:{target_epsg} (Brazil Polyconic) e salvando...")
gdf_final = gdf_calc.to_crs(epsg=target_epsg)

# Limpeza opcional de colunas intermediárias para o arquivo não ficar gigante
# cols_drop = ['e_des_pct', 'e_inu_pct', 'e_inu_abs', 'mean']
# gdf_final = gdf_final.drop(columns=cols_drop, errors='ignore')

gdf_final.to_file(output_gpkg_final, driver='GPKG')
print(f"Processo finalizado! Arquivo salvo em: {output_gpkg_final}")

1. Carregando malha do passo anterior...
2. Verificando CRS do Raster de Inundações...
   CRS do Raster: EPSG:4326
3. Calculando área inundada...
4. Ponderando pela densidade de domicílios...
5. Normalizando indicador de inundações...
6. Revertendo para EPSG:5880 (Brazil Polyconic) e salvando...
Processo finalizado! Arquivo salvo em: ../data/clean/BR_setores_CD2022_e125_v1234_i1234_norm.gpkg


## 2/2 -- Inputar dados do MapBiomas no H3 usando DuckDB

### A) Inputar dados de deslizamentos do Mapbiomas no .parquet do H3

In [ ]:
import duckdb
import rasterio
import pandas as pd

# --- CONFIGURAÇÕES ---
CAMINHO_H3_INPUT = '../data/clean/h3/br/br_h3_res9_com_setores.parquet'
RASTER_DESLIZAMENTOS = '../data/raw/mapbiomas/mapbiomas_landslides_2024.tif'
ARQUIVO_SAIDA = '../data/clean/h3/br/br_h3_res9_e1_deslizamento.parquet'

# 1. Conectar ao DuckDB
con = duckdb.connect()

# Tenta instalar do repositório da comunidade (onde o H3 vive hoje)
try:
    con.install_extension("h3", repository="community")
    con.load_extension("h3")
except Exception as e:
    print(f"Erro ao carregar H3: {e}")
    # Fallback caso a versão comunidade falhe:
    con.sql("SET custom_extension_repository='http://extensions.duckdb.org/community';")
    con.sql("INSTALL h3;")
    con.load_extension("h3")

print("--- PROCESSANDO COORDENADAS COM DUCKDB ---")

# 2. Extrair Lat/Lon usando SQL (Muito mais rápido que loop h3.cell_to_latlng)
# h3_cell_to_lat retorna um struct {lat, lng}
df_coords = con.sql(f"""
    SELECT 
        *,
        h3_cell_to_lat(h3_id) as lat,
        h3_cell_to_lng(h3_id) as lon
    FROM read_parquet('{CAMINHO_H3_INPUT}')
""").df()

# 3. Amostragem do Raster (Rasterio continua sendo a melhor ferramenta aqui)
print("Amostrando raster...")
with rasterio.open(RASTER_DESLIZAMENTOS) as src:
    # Passamos as colunas do dataframe diretamente para o gerador de amostras
    # Invertemos para (lon, lat) conforme exigido pelo rasterio para EPSG:4326
    coord_samples = zip(df_coords['lon'], df_coords['lat'])
    
    # Atribuição direta (o list comprehension aqui é rápido)
    df_coords['e_des_val'] = [val[0] for val in src.sample(coord_samples)]

# 4. Normalização com SQL (DuckDB é excelente para cálculos matemáticos)
# Registramos o dataframe como uma tabela temporária para usar SQL de novo
con.register('df_final', df_coords)

print("Finalizando cálculos e salvando...")
con.sql("""
    COPY (
        SELECT 
            *,
            CASE 
                WHEN max(e_des_val) OVER () > 1 
                THEN (e_des_val - min(e_des_val) OVER ()) / (NULLIF(max(e_des_val) OVER () - min(e_des_val) OVER (), 0))
                ELSE e_des_val 
            END as e_des_norm
        FROM df_final
    ) TO '{ARQUIVO_SAIDA}' (FORMAT PARQUET)
""".format(ARQUIVO_SAIDA=ARQUIVO_SAIDA))

print("Concluído!")

--- PROCESSANDO COORDENADAS COM DUCKDB ---
Amostrando raster...
Finalizando cálculos e salvando...
Concluído!


### B) Inputar dados de inundações do Mapbiomas no .parquet do H3

In [ ]:
import duckdb
import rasterio
import pandas as pd
import os

# --- CONFIGURAÇÕES ---
CAMINHO_H3_INPUT = '../data/clean/h3/br_h3_res9_com_setores.parquet'
RASTER_INUNDACOES = '../data/raw/mapbiomas/mapbiomas_floods_2024.tif'
ARQUIVO_SAIDA = '../data/clean/h3/br_h3_res9_e2_inundacao.parquet'

# 1. Conexão e Configuração da Extensão H3
con = duckdb.connect()

print("--- CONFIGURANDO EXTENSÃO H3 ---")
# Ajuste vital para evitar o erro 404 no Windows
con.sql("SET custom_extension_repository='http://extensions.duckdb.org/community';")
con.sql("INSTALL h3; LOAD h3;")

print("--- EXTRAINDO COORDENADAS COM DUCKDB ---")

# 2. SQL para Lat/Lon
# O DuckDB processa os 5 milhões de registros muito mais rápido que o Python puro
df_temp = con.sql(f"""
    SELECT 
        h3_id, 
        qtd_enderecos,
        h3_cell_to_lat(h3_id) as lat,
        h3_cell_to_lng(h3_id) as lon
    FROM read_parquet('{CAMINHO_H3_INPUT}')
""").df()

# 3. Amostragem do Raster (Rasterio)
print("Amostrando raster de inundações...")
with rasterio.open(RASTER_INUNDACOES) as src:
    # Gerador eficiente para não estourar a RAM
    coord_samples = zip(df_temp['lon'], df_temp['lat'])
    df_temp['e_inu_val'] = [val[0] for val in src.sample(coord_samples)]

# 4. Cálculo do Índice e Normalização no DuckDB
con.register('dados_amostrados', df_temp)

print("Calculando ponderação (endereços) e normalização...")
# Usamos SQL para cálculos de coluna (vetorizados)
con.sql(f"""
    COPY (
        WITH calculo_base AS (
            SELECT 
                *,
                -- Multiplica a presença de inundação pela quantidade de endereços
                COALESCE(e_inu_val, 0) * qtd_enderecos AS e_inu_abs
            FROM dados_amostrados
        )
        SELECT 
            *,
            -- Normalização Min-Max (0 a 1) sobre o valor absoluto
            (e_inu_abs - min(e_inu_abs) OVER ()) / 
            NULLIF(max(e_inu_abs) OVER () - min(e_inu_abs) OVER (), 0) AS e_inu_norm
        FROM calculo_base
    ) TO '{ARQUIVO_SAIDA}' (FORMAT PARQUET)
""")

print(f"Concluído! Arquivo salvo com sucesso em: {ARQUIVO_SAIDA}")

--- CONFIGURANDO EXTENSÃO H3 ---
--- EXTRAINDO COORDENADAS COM DUCKDB ---
Amostrando raster de inundações...
Calculando ponderação (endereços) e normalização...
Concluído! Arquivo salvo com sucesso em: ../data/clean/h3/br_h3_res9_exposicao_inundacao.parquet
